In [1]:
import os
import cv2
import random
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam

2022-12-13 14:16:07.086276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 14:16:07.179228: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-13 14:16:07.626256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/fpserver/anaconda3/envs/tf/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda-11.7/lib64:
2022-12-13 14:16:07.626302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load 

In [2]:
reshape_size = (256,256)
root_dir = "/media/fpserver/New Volume/DATASETS/van_gogh/VincentVanGogh"

i = 0
for dir in os.listdir(root_dir):
  for imgfile in os.listdir(os.path.join(root_dir, dir)):
    
    img = cv2.imread(root_dir + "/" + dir + "/" + imgfile)
    if img is None:
      print("wrong path")
    else:
      img = cv2.resize(img, reshape_size)
      cv2.imwrite(f"/media/fpserver/New Volume/DATASETS/van_gogh/resized/{i}.png", img)
    i += 1

In [2]:
len(os.listdir("/media/fpserver/New Volume/DATASETS/van_gogh/resized"))

2025

In [3]:
#parameters

img_width = 256
img_height = 256
channels = 3
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(0.0001)

In [4]:
#generator
def build_generator():
  model = Sequential()

  model.add(Dense(256 * 32 * 32, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((32,32,256)))

  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(3, (3,3), activation="tanh", padding="same"))

  model.summary()

  return model

generator = build_generator()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 262144)            26476544  
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 262144)            0         
                                                                 
 reshape (Reshape)           (None, 32, 32, 256)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 64, 64, 128)      524416    
 nspose)                                                         
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 128)       0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 128, 128, 128)    262272    
 ranspose)                                              

2022-12-13 14:16:22.687326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:16:22.728032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:16:22.728175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 14:16:22.729044: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [5]:
def build_discriminator():
  model = Sequential()
  model.add(Conv2D(64, (3,3), padding="same", input_shape=img_shape))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(128, (3,3), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(256, (3,3), padding="same"))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Flatten())
  model.add(Dropout(0.4))
  model.add(Dense(1, activation="sigmoid"))

  model.summary()

  return model

discriminator = build_discriminator()
discriminator.compile(loss="binary_crossentropy", optimizer=adam, metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 leaky_re_lu_4 (LeakyReLU)   (None, 256, 256, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 256, 256, 128)     73856     
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 256, 256, 128)     0         
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 128)     147584    
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 256, 256, 128)     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 256)    

In [6]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)
GAN.compile(loss="binary_crossentropy", optimizer=adam)

In [7]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 2, 2
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001

    # Rescale images 0 - 1
    gen_imgs = (gen_imgs + 1) / 2.0

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    #fig.savefig("currentgeneration.png")
    fig.savefig("/media/fpserver/New Volume/DATASETS/van_gogh/generated/%.8f.png" % save_name)
    plt.close()

In [8]:
from PIL import Image

def train(epochs, batch_size=32, save_interval=200):

  array = []
  #PUT PATH OF RESIZED IMAGES
  path = "/media/fpserver/New Volume/DATASETS/van_gogh/some/"

  for dir in os.listdir(path):
            # print(dir)
    image = Image.open(path + dir)
    data = np.asarray(image)
    array.append(data)

  X_train = np.array(array)
  print(X_train.shape)

  # print(X_train.shape)
  #Rescale data between -1 and 1
  X_train = X_train / 127.5 -1.
  bat_per_epo = int(X_train.shape[0] / batch_size)
  # X_train = np.expand_dims(X_train, axis=3)
  print(X_train.shape)

  #Create our Y for our Neural Networks
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    for j in range(bat_per_epo):
      #Get Random Batch
      idx = np.random.randint(0, X_train.shape[0], batch_size)
      imgs = X_train[idx]

      #Generate Fake Images
      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      gen_imgs = generator.predict(noise)

      #Train discriminator
      d_loss_real = discriminator.train_on_batch(imgs, valid)
      d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

      noise = np.random.normal(0, 1, (batch_size, latent_dim))
      
      #inverse y label
      g_loss = GAN.train_on_batch(noise, valid)

      print("******* %d %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch,j, d_loss[0], 100* d_loss[1], g_loss))

      # if(epoch % save_interval) == 0:
    save_imgs(epoch)


train(10000, batch_size=32, save_interval=200)

(517, 256, 256, 3)
(517, 256, 256, 3)


2022-12-13 14:17:12.014489: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-12-13 14:17:12.128556: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


1/1 [==============================] - 7s 7s/step
******* 0 0 [D loss: 0.721866, acc: 17.19%] [G loss: 0.721464]
1/1 [==============================] - 0s 10ms/step
******* 0 1 [D loss: 0.302137, acc: 100.00%] [G loss: 0.913616]
1/1 [==============================] - 0s 10ms/step
******* 0 2 [D loss: 0.219393, acc: 100.00%] [G loss: 1.234724]
1/1 [==============================] - 0s 10ms/step
******* 0 3 [D loss: 0.132575, acc: 100.00%] [G loss: 1.787550]
1/1 [==============================] - 0s 10ms/step
******* 0 4 [D loss: 0.061687, acc: 100.00%] [G loss: 2.639456]
1/1 [==============================] - 0s 10ms/step
******* 0 5 [D loss: 0.022103, acc: 100.00%] [G loss: 3.751183]
1/1 [==============================] - 0s 9ms/step
******* 0 6 [D loss: 0.006609, acc: 100.00%] [G loss: 5.005924]
1/1 [==============================] - 0s 10ms/step
******* 0 7 [D loss: 0.001866, acc: 100.00%] [G loss: 6.278381]
1/1 [==============================] - 0s 9ms/step
******* 0 8 [D loss: 0.05

KeyboardInterrupt: 